In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torchmetrics import MeanSquaredError, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [3]:
# colab i kaggle jeszcze nie testowane
colab = False
kaggle = False
test_number = '12_2'

In [4]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'
checkpoints = 'checkpoints'
if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    dataset = '/content/drive/MyDrive/MGU/cifar10/'
    
    color_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{color_imgs}'
    gray_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{gray_imgs}'
    checkpoints = f'/content/drive/MyDrive/MGU/{test_number}/{checkpoints}'
elif kaggle:
    dataset = '/kaggle/input/cifar10/'
    
    color_imgs = f'{test_number}/{color_imgs}'
    gray_imgs = f'{test_number}/{gray_imgs}'
    checkpoints = f'{test_number}/{checkpoints}'
else:
    dataset = '../../datasets/cifar10/'

In [5]:
# Make folders and set parameters
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = [1e10, 1e10, 1e10]
best_epoch = -1
patience = 50
epochs = 500
batch_size = 128
SIZE = 32

In [6]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [7]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [8]:
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(),  
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE), 
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        return img_gray, img_ab
    
    def __len__(self):
        return len(self.paths)

In [9]:
# Training
train_imagefolder = LabImageFolder(dataset + 'train')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder(dataset + 'val' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [10]:
kernel_size=3
stride_en=1
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'
channels_base = 16

class Autoencoder(nn.Module):
    def __init__(self, input_size=128):
        super(Autoencoder, self).__init__()

        self.conv1 = nn.Conv2d(1, channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        self.conv2 = nn.Conv2d(channels_base, channels_base * 2, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)

        self.convtrans1 = nn.ConvTranspose2d(channels_base * 2, channels_base, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans2 = nn.ConvTranspose2d(channels_base, channels_base // 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans3 = nn.ConvTranspose2d(channels_base // 2, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(channels_base)
        self.batchnorm2 = nn.BatchNorm2d(channels_base * 2)
        self.batchnorm3 = nn.BatchNorm2d(channels_base)
        self.batchnorm4 = nn.BatchNorm2d(channels_base // 2)
        
    def forward(self, input):
        # encoder
        x = nn.LeakyReLU(0.1)(self.batchnorm1(self.conv1(input)))
        x = nn.AvgPool2d(kernel_size=3, stride=2, padding=1)(x)
        x = nn.LeakyReLU(0.1)(self.batchnorm2(self.conv2(x)))
        x = nn.AvgPool2d(kernel_size=3, stride=2, padding=1)(x)
        
        # decoder
        x = nn.LeakyReLU(0.1)(self.batchnorm3(self.convtrans1(x)))
        x = nn.Upsample(scale_factor=scale_factor)(x)
        x = nn.LeakyReLU(0.1)(self.batchnorm4(self.convtrans2(x)))
        x = nn.Upsample(scale_factor=scale_factor)(self.convtrans3(x))

        return x

In [11]:
model = Autoencoder()

In [12]:
criterion = [MeanSquaredError(), PeakSignalNoiseRatio(data_range=1.0), StructuralSimilarityIndexMeasure(data_range=1.0)]

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [14]:
# # Move model and loss function to GPU
if use_gpu: 
    criterion = [criterion[0].to("cuda"), criterion[1].to("cuda"), criterion[2].to("cuda")]
    model = model.cuda()

In [15]:
if use_gpu: 
    from torchsummary import summary
    summary(model, (1, SIZE, SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             160
       BatchNorm2d-2           [-1, 16, 32, 32]              32
            Conv2d-3           [-1, 32, 16, 16]           4,640
       BatchNorm2d-4           [-1, 32, 16, 16]              64
   ConvTranspose2d-5             [-1, 16, 8, 8]           4,624
       BatchNorm2d-6             [-1, 16, 8, 8]              32
   ConvTranspose2d-7            [-1, 8, 16, 16]           1,160
       BatchNorm2d-8            [-1, 8, 16, 16]              16
   ConvTranspose2d-9            [-1, 2, 16, 16]             146
Total params: 10,874
Trainable params: 10,874
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.43
Params size (MB): 0.04
Estimated Total Size (MB): 0.47
---------------------------------------------

In [16]:
class AverageMeter(object):
    '''A handy class from the PyTorch ImageNet tutorial''' 
    def __init__(self):
        self.reset()
    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
    '''Show/save rgb image from grayscale and ab channels
       Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    plt.clf() # clear matplotlib 
    color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    if save_path is not None and save_name is not None: 
        plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
        plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [17]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]

    model.eval()
    already_saved_images = False
    for gray, ab in val_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()

        # Run model and record loss
        output_ab = model(gray) # throw away class predictions
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print(f'Validate: MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print('Finished validation.')
    if epoch >= 0:
        writer.add_scalar("MSE/test", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/test", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/test", _loss[2].avg, epoch)
    return _loss[0].avg, _loss[1].avg, _loss[2].avg

In [18]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]
    
    model.train()

    for gray, ab in train_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray) 
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        loss[0].backward()
        optimizer.step()
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))
        
    print(f'Epoch: {epoch}, MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("MSE/train", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/train", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/train", _loss[2].avg, epoch)

In [19]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses[0] < best_losses[0]:
        best_losses[0] = losses[0]
        best_epoch = epoch
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}.pth')
    if losses[1] < best_losses[1]:
        best_losses[1] = losses[1]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-PSNRLoss-{losses[1]:.8f}.pth')
    if losses[2] < best_losses[2]:
        best_losses[2] = losses[2]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-SSIMLoss-{losses[2]:.8f}.pth')
    
    if epoch - best_epoch >= patience and epoch >= 100:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}-early_stop.pth')
        break
    
    if epoch == epochs - 1:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')


Starting training epoch 0
Epoch: 0, MSE 0.00266816 (0.00852265), PSNR 25.73788261 (23.95481434), SSIM 0.69754821 (0.62294808)
Finished training epoch 0
Validate: MSE 0.00330356 (0.00288826), PSNR 24.81017685 (25.45355681), SSIM 0.64724749 (0.71157628)
Finished validation.
Starting training epoch 1
Epoch: 1, MSE 0.00299765 (0.00287464), PSNR 25.23219490 (25.43721078), SSIM 0.74105930 (0.72826823)
Finished training epoch 1
Validate: MSE 0.00303550 (0.00288348), PSNR 25.17769241 (25.47755137), SSIM 0.67512870 (0.73978443)
Finished validation.
Starting training epoch 2
Epoch: 2, MSE 0.00266213 (0.00272855), PSNR 25.74770927 (25.66260086), SSIM 0.76227403 (0.74803741)
Finished training epoch 2
Validate: MSE 0.00315067 (0.00272202), PSNR 25.01597404 (25.71591089), SSIM 0.68909752 (0.75572054)
Finished validation.
Starting training epoch 3
Epoch: 3, MSE 0.00286351 (0.00265088), PSNR 25.43101120 (25.78710650), SSIM 0.76057923 (0.75839802)
Finished training epoch 3
Validate: MSE 0.00290070 (0.0

Epoch: 30, MSE 0.00255100 (0.00235736), PSNR 25.93290138 (26.29773726), SSIM 0.77571046 (0.77742345)
Finished training epoch 30
Validate: MSE 0.00265663 (0.00268701), PSNR 25.75668716 (25.82416475), SSIM 0.71697187 (0.77469939)
Finished validation.
Starting training epoch 31
Epoch: 31, MSE 0.00231129 (0.00235783), PSNR 26.36145210 (26.29835380), SSIM 0.77514809 (0.77739112)
Finished training epoch 31
Validate: MSE 0.00276290 (0.00238401), PSNR 25.58634758 (26.29540966), SSIM 0.71823829 (0.77621649)
Finished validation.
Starting training epoch 32
Epoch: 32, MSE 0.00185193 (0.00235846), PSNR 27.32374954 (26.29670455), SSIM 0.79071057 (0.77732346)
Finished training epoch 32
Validate: MSE 0.00255652 (0.00291746), PSNR 25.92350578 (25.49784443), SSIM 0.71515667 (0.76914704)
Finished validation.
Starting training epoch 33
Epoch: 33, MSE 0.00186745 (0.00237328), PSNR 27.28750229 (26.26988641), SSIM 0.76210666 (0.77719486)
Finished training epoch 33
Validate: MSE 0.00277937 (0.00238806), PSNR 

Epoch: 60, MSE 0.00207256 (0.00231756), PSNR 26.83493614 (26.37125578), SSIM 0.80636060 (0.77758245)
Finished training epoch 60
Validate: MSE 0.00268787 (0.00260056), PSNR 25.70591545 (25.93756345), SSIM 0.71618950 (0.77207086)
Finished validation.
Starting training epoch 61
Epoch: 61, MSE 0.00246391 (0.00231681), PSNR 26.08375168 (26.37372702), SSIM 0.75795603 (0.77743900)
Finished training epoch 61
Validate: MSE 0.00258565 (0.00230679), PSNR 25.87429810 (26.43189297), SSIM 0.71773982 (0.77432863)
Finished validation.
Starting training epoch 62
Epoch: 62, MSE 0.00235049 (0.00232106), PSNR 26.28840637 (26.36544965), SSIM 0.75764430 (0.77752179)
Finished training epoch 62
Validate: MSE 0.00311326 (0.00262346), PSNR 25.06784821 (25.89420192), SSIM 0.71451879 (0.77886633)
Finished validation.
Starting training epoch 63
Epoch: 63, MSE 0.00289834 (0.00230690), PSNR 25.37850189 (26.39374256), SSIM 0.75241822 (0.77771352)
Finished training epoch 63
Validate: MSE 0.00310160 (0.00280540), PSNR 

Epoch: 90, MSE 0.00274559 (0.00228531), PSNR 25.61363983 (26.43286851), SSIM 0.76242614 (0.77788184)
Finished training epoch 90
Validate: MSE 0.00273018 (0.00254634), PSNR 25.63808250 (26.01281944), SSIM 0.71925521 (0.76907373)
Finished validation.
Starting training epoch 91
Epoch: 91, MSE 0.00186288 (0.00229254), PSNR 27.29815865 (26.41933986), SSIM 0.79850370 (0.77785018)
Finished training epoch 91
Validate: MSE 0.00266171 (0.00301244), PSNR 25.74839020 (25.35710573), SSIM 0.71305096 (0.76760290)
Finished validation.
Starting training epoch 92
Epoch: 92, MSE 0.00240153 (0.00228867), PSNR 26.19511223 (26.42750604), SSIM 0.76864231 (0.77786613)
Finished training epoch 92
Validate: MSE 0.00264045 (0.00234538), PSNR 25.78322220 (26.36210355), SSIM 0.71976101 (0.77598456)
Finished validation.
Starting training epoch 93
Epoch: 93, MSE 0.00206728 (0.00228917), PSNR 26.84600639 (26.42416420), SSIM 0.78888017 (0.77779250)
Finished training epoch 93
Validate: MSE 0.00268777 (0.00242971), PSNR 

Epoch: 120, MSE 0.00237000 (0.00228191), PSNR 26.25252342 (26.43999954), SSIM 0.74292231 (0.77802290)
Finished training epoch 120
Validate: MSE 0.00267785 (0.00233681), PSNR 25.72213745 (26.37274673), SSIM 0.71099436 (0.77639191)
Finished validation.
Starting training epoch 121
Epoch: 121, MSE 0.00235508 (0.00228205), PSNR 26.27993774 (26.43903217), SSIM 0.78321737 (0.77817371)
Finished training epoch 121
Validate: MSE 0.00261695 (0.00232077), PSNR 25.82203674 (26.40963401), SSIM 0.72228581 (0.77598435)
Finished validation.
Starting training epoch 122
Epoch: 122, MSE 0.00236579 (0.00228019), PSNR 26.26023102 (26.44171072), SSIM 0.77581131 (0.77790868)
Finished training epoch 122
Validate: MSE 0.00264401 (0.00235883), PSNR 25.77736855 (26.33730253), SSIM 0.72006404 (0.77860753)
Finished validation.
Starting training epoch 123
Epoch: 123, MSE 0.00248021 (0.00228193), PSNR 26.05512047 (26.43922981), SSIM 0.76868993 (0.77784846)
Finished training epoch 123
Validate: MSE 0.00278129 (0.00235

Validate: MSE 0.00273965 (0.00336190), PSNR 25.62305069 (24.93524029), SSIM 0.71116298 (0.75989435)
Finished validation.
Starting training epoch 150
Epoch: 150, MSE 0.00220909 (0.00227122), PSNR 26.55785561 (26.46255027), SSIM 0.79041135 (0.77808971)
Finished training epoch 150
Validate: MSE 0.00285251 (0.00245262), PSNR 25.44772339 (26.19253291), SSIM 0.71549034 (0.77808792)
Finished validation.
Starting training epoch 151
Epoch: 151, MSE 0.00265091 (0.00226801), PSNR 25.76605225 (26.46682872), SSIM 0.76581115 (0.77795261)
Finished training epoch 151
Validate: MSE 0.00265929 (0.00234523), PSNR 25.75234795 (26.36502236), SSIM 0.71815610 (0.77827109)
Finished validation.
Starting training epoch 152
Epoch: 152, MSE 0.00195206 (0.00226467), PSNR 27.09505653 (26.47332381), SSIM 0.79295599 (0.77808384)
Finished training epoch 152
Validate: MSE 0.00275756 (0.00228591), PSNR 25.59474564 (26.46724285), SSIM 0.71321583 (0.77622787)
Finished validation.
Starting training epoch 153
Epoch: 153, MS

<Figure size 432x288 with 0 Axes>

In [20]:
torch.save(model.state_dict(), f'{checkpoints}/last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')

In [21]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

Validate: MSE 0.00274313 (0.00230722), PSNR 25.61752701 (26.43065089), SSIM 0.71537215 (0.77627632)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [22]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()